In [1]:
%%javascript
MathJax.Hub.Config({TeX: { equationNumbers: { autoNumber: "AMS" } }});

<IPython.core.display.Javascript object>

# BMWFLC - band-limited multiple-WFLC

In this section, I will propose a new method for tracking multiple frequencies in a pathological tremor signal. The proposed filter is a combination of the E-BMFLC filter and the WFLC filter. Like the band-limited multiple-FLC (BMFLC) filter that consists of multiple-FLC, my idea is to extend this to a band-limited multiple-WFLC (BMWFLC) filter. In int he first figure we can see the new filter architecture, we see that $n$-WFLC are combined to form the BMWFLC. The novelty here is that each frequency in the band now can adjust to the signal.  To accomplish this we measure the magnitude of the Fourier coefficients of each WFLC in the BMWFLC, the WFLC with the largest magnitude contains the dominant frequency in the estimated signal. It is now possible to measure all the frequencies present in the estimated signal, and their strength with respect to each other.


## Core Equations

Each $n$-WFLC in the BMWFLC has its own frequency and learning rate, enabling each WFLC to adapt to the signal individually with its own speed. The learning rates are

\begin{equation}\label{eq:BMWFLClearningrates}
\boldsymbol{\mu}_k= \begin{bmatrix}
\mu_{1k}&\mu_{2_k}&\cdots&\mu_{n_k}
\end{bmatrix}
\end{equation}

and the angular frequencies for the WFLC in the filter are

\begin{equation}\label{eq:BMWFLCfrequencies}
\boldsymbol{\omega}_k= \begin{bmatrix}
\omega_{1_k}&\omega_{2_k}&\cdots&\omega_{n_k}
\end{bmatrix}
\end{equation}

where $k$ is the sampling instance. When $k=0$ we get the size frequency window, where the frequencies have the distance $\Delta w$  between them 

\begin{equation}\label{eq:BWMFLCwindow}
[\omega_{1_0},\omega_{n_0}]
\end{equation}

The frequency window is divided into a number of divisions 

\begin{equation}\label{eq:divisions}
n=\frac{\omega_{n_0}-\omega_{1_0}}{\Delta \omega}
\end{equation}

and $n$ is then the number of harmonics in for the Fourier combiner model of $y_k$. 

\begin{equation}
y_k=\sum_{r=1}^{n}a_{rk}\sin(\omega_{r_k}k )+b_{rk}\cos(\omega_{r_k}k)
\end{equation}

$y_k$ will not be used as an output for this filter, since it will not be tuned to have an accurate estimate of the amplitude, only the frequencies  \eqref{eq:BWMFLCwindow} are of interest. Then $\textbf{x}_k$ and $\textbf{w}_k$ are defined

\begin{align}
\textbf{x}_k&=
\begin{bmatrix}
[\sin(\omega_{1_k}k)&\sin(\omega_{2_k}k)&\cdots&\sin(\omega_{n_k}k)]^T \\ \label{eq:BMWLFCx}
[\cos(\omega_{1_k}k)&\cos(\omega_{2_k}k)&\cdots&\cos(\omega_{n_k}k)]^T
\end{bmatrix}\\ \label{eq:BMWLFCw}
\textbf{w}_k&=\begin{bmatrix}
[a_{1k}&a_{2k}&\cdots & a_{nk}]^T\\
[b_{1k}&b_{2k}&\cdots & b_{nk}]^T
\end{bmatrix} 
\end{align}

Using \eqref{eq:BMWLFCx} and \eqref{eq:BMWLFCw} the linear combiner can be written as

\begin{equation}\label{eq:error_sy}
y_k=\textbf{w}_k^T\textbf{x}_k
\end{equation}

and the error beween the signal of the complete motion of the hand and the estimated signal $y_k$ is

\begin{equation}\label{key}
\varepsilon_k=s_k-y_k
\end{equation}

The recursive [least mean squares (LMS) algorithm](https://en.wikipedia.org/wiki/Least_mean_squares_filter) used to update the weights of $\textbf{w}_k$

\begin{align}\label{}
\textbf{w}_{k+1}&=\rho \textbf{w}_{k}+2\mu_k\textbf{x}_{k}\varepsilon_k \label{eq:LMS}\\
\rho&=\sqrt[\delta]{\alpha}\\
\delta&=\frac{1}{\Delta T}T_p\label{}
\end{align}

This is the *memory manipulation* from the E-BMFLC algorithm in the [E-BMFLC notebook](5.%20E-BMFLC.ipynb). It makes the filter forget past information, making it adapt faster when sudden changes occur in the signal $s_k$, E.g., a sudden change of frequency.
Decreasing $\rho$ increases the rate of forgetting past information. $\Delta T$ is the sampling time (in seconds), $T_p$ is the width of the considered memory window (in seconds), and $\alpha$ is the minimum amplification gain considered within the window.

The fundamental angular frequencies $\boldsymbol{\mu}_k$, are updated using multiple modified LSM algorithms, one for each WFLC, each with its own adaptive gain $\boldsymbol{\omega}_k$

\begin{equation}\label{eq:wrk}
\omega_{r_{k+1}}=\omega_{r_k}+2\varepsilon_k\mu_{rk}\left[a_{rk}\cos(r\omega_{r_k}k)-b_{rk}\sin(\omega_{r_k}k) \right], \hspace{16pt} r=1,2,\ldots, n
\end{equation}

Equation \eqref{eq:wrk} is a special case of Equation (7) with one harmonic from the [WFLC notebook](3.%20WFLC.ipynb). Equation  \eqref{eq:wrk} uses the [LMS algorithm](https://en.wikipedia.org/wiki/Least_mean_squares_filter) to descend the performance surface to minimize the [mean squared error (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error). If the learning rates $\boldsymbol{\mu}_k$, are too large all the frequencies will try to reach the frequencies with the largest magnitude in the magnitude spectrum, the global minimum on the performance surface. This is because the error $\varepsilon_{k}$ looks at the error between the input signal $s_k$ and the estimate of the entire signal $y_k$, so each WFLC in the BMWFLC wants to go for the dominant frequency in the signal. If the learning rates $\boldsymbol{\mu}_k$, is set small enough, each WFLC will get stuck in its local minimum, this is what we want, enabling each WFLC only converge towards the real frequency closest to it. When tracking multiple frequencies, the learning rate needs to be small, resulting in slow convergence, if only the frequency with the largest magnitude is being tracked, the learning rate can be set much higher since only the global minimum is of interest, resulting in faster convergence. 

![bmwflc](fig/bmwflc.png)

## Adaptive Learning Rate

One of the drawbacks with the FLC, WFLC and BMFLC algorithms is that the learning rate parameters $\mu$ and $\mu_0$ needs fine tuning, and are sensitive to change in the amplitude of the input signal \cite{popovic2010adaptive}. The two parameters also need to be tuned separately. Our primary goal is to dampen the tremor for people with PD and ET, but when the tremor is suppressed the amplitude of the signal will be decreased, and the sensitivity of the estimation will decrease. To solve this problem, we can use the amplitude of the input signal to adapt the learning rate. I have come up with a simple but elegant solution to this problem. We will use a queue data structure $Q$, which uses the first-in, first-out, or FIFO, policy. The queue will have a fixed length $L$, a good value for its length is to set it equal to the sampling rate of the signal, $f_s$. 

![bmwflc-band](fig/algorithm1.png)

Algorithm 1 checks the $L$ last samples from the tremor signal, and return the peak amplitude. This value can now be used to adapt the learning rates. If the length $L$ is set to $100$ and our sample rate of the signal is $100$ Hz, the window will contain the tremor values for the past 1 second so that the learning rates can adapt relatively fast to any changes in the amplitude of the tremor. The following equations adapt the learning rates.

\begin{equation}\label{eq:Adaptive learningrate}
\mu_k = \dfrac{\kappa}{A_{peak}}
\end{equation}

The value for the scaling parameter $\kappa$ can be found by tuning, I found that a good starting value is $0.01$. The equation to make $\mu_{nk}$  adaptive is

\begin{equation}\label{eq:adaptlearningrate}
\mu_{rk} = \mu_k h + \mu_{rk}\beta e^{-\lambda t}, \hspace{16pt} r=1,2,\ldots, n
\end{equation}

For the first term $h$ scales the learning rate $u_{nk}$ with respect to the value of $\mu_k$. The second term is adds exponential decay, making $mu_{r_k}$  an exponential decaying learning rate. At time $t=0$, $e^0=1$, and $\mu_{r_k} = \mu_k h + \mu_{r_k}\beta $. When $\lim\limits_{t\to \infty}$ the term becomes $0$, and $\mu_{nk} = \mu_k h$ . What this does is to make the learning rate larger in the start and exponentially decaying to the desired and stable $\mu_{r_k} = \mu_k h 
$ form.  
$\mu_{r_k}$ should have a value that is $\mu_{r_k}<<\mu_k$ for it to be stable ([Adaptive band-pass filter (ABPF) for tremor extraction from inertial sensor data](https://www.sciencedirect.com/science/article/pii/S0169260710000702)),  so for the scaling parameter $h$ a good starting value is $0.00001$. The parameter $\beta$ is the value we want scale up the learnngrate at the start, a good value to start with is 50. $\lambda$ (known as the decay constant) makes the learning rate decay faster when it increases i size. A good starting value for this constant is 0.2. The reason to use a exponential decaying learning rate is to make the estimation of the frequency go faster in the start, but for such high values, the filter would become unstable over time, so it decays to a stable value. 

## Magnitude Spectrum - Extracting Frequencies
To find the frequencies in the input signal, we will look at the magnitudes of the Fourier components in the BMFLC, the magnitude of each FLC; the FLC with the highest magnitude should contain the dominant frequency in the estimated signal, which then is the estimate of the dominant frequency for the real signal. This is based on the concept of magnitude spectrum for a Fourier series, presented in the [Fourier Series notebook](1.%20Fourier%20Series.ipynb). 

\begin{equation}\label{eq:magnitude_BMWFLC}
M_{rk}=\sqrt{a_{rk}^2 + b_{rk}^2},  \hspace{16pt} r=1,2,\ldots, n
\end{equation}
\begin{equation}\label{eq:magnitude_array}
\textbf{M}=\begin{bmatrix}
M_{1k}&M_{2k}&\cdots&M_{nk}
\end{bmatrix}
\end{equation}

The cool thing about this method is that we can use it to estimate multiple frequencies in the signal. To accomplish this, we look for the magnitudes that are peaks in the magnitude spectrum.   By peak magnitude I mean that the magnitude of interest should be larger than the magnitude to its immediate left and right in the magnitude spectrum, $M_{r-1} < M_r > M_{r+1}$, an example can be seen in Figure \ref{fig:BMWFLCmagnitudeSepctrum}, where the two red markers are peaks in the spectrum. We  want to be able to tell our algorithm how many peaks it should track in the spectrum, each peak it tracks is an estimate of a frequency, so we define a variable called $ftt$ (frequencies to track). Instead of making all the FLC in the truncated Fourier series into WFLC, we can say that the $ftt$ number of FLC with the largest magnitude of Fourier coefficients are allowed to become WFLC. When the learning rate is set to zero, $\mu_{rk}=0$, it becomes an FLC, and its respective frequency $\omega_{r_k}$ lose its capability to adapt to the signal. When the BWMFLC starts to adapt to the signal, all the magnitudes of the FLC are tracked, and the $fft$ number of FLC with the largest peak magnitudes in the magnitude spectrum become WFLC, by setting $\mu_{rk} > 0$. We define an array to store the peak magnitudes and their position in the spectrum.  $M_{peak_1}$ is largest, $M_{peak_2}$ second largest peaks and $Pos_{p_1}$ and $Pos_{p_2}$ are their position in the spectrum respectively. The magnitude peaks can now be used to compare how much power the different dominant frequencies have in respect to each other, this is not something I have implemented here, so this should be looked further at in future iterations of the BMWFLC filter. 

\begin{equation}\label{eq:MagPeak}
\textbf{M}_{peak}=\begin{bmatrix}
[M_{p_1},Pos_{p_1}]&[M_{p_2},Pos_{p_2}]&\cdots&[M_{p_r}, Pos_{p_r}]
\end{bmatrix}
\end{equation}

Algorithm 2 shows how we can find the magnitude peaks, and return an array with all the peaks, it should be sorted the same way as \eqref{eq:MagPeak}.

![bmwflc-band](fig/algorithm2.png)

It is now easy to get the estimate for the frequencies; we use the positions from \eqref{eq:MagPeak}, and the estimates will be the frequencies in \eqref{eq:BWMFLCwindow} that corresponds to them. E.g if $Pos_{p_1} = 8$, the estimate for the frequency with the most power in the input signal will be $\omega_{8_k}$. In Figure \ref{fig:BMWFLC_band} we see the frequency window of the BMWFLC, the two arrows with red arrowheads represent the two dominant frequencies in the signal when $ftt=2$, they can adapt to the signal by setting the learning rates, $\mu_{4k} > 0$ and $\mu_{7k} > 0$. 

![bmwflc-band](fig/algorithm3.png)\label{alg:maxAmp}

Since the frequencies in $\boldsymbol{\omega}_k$ will move change over time, there should be a mechanism to reset the frequencies to their original values $\boldsymbol{\omega}_0$, if they no longer are present in the signal. An easy way to do this is to make a threshold variable, $\eta$, and if any of the magnitudes in $\textbf{M}_k$ gets under this threshold, its respective frequency in $\boldsymbol{\omega}_k$  should reset to its original value $\boldsymbol{\omega}_0$. 

\begin{equation}\label{eq:threshold}
\omega_{r_k} = 
\begin{cases}
\omega_{r_0},& \text{if } M_{rk} <  \eta\\
\omega_{r_k} ,              & \text{otherwise}
\end{cases} \hspace{16pt} r=1,2,\ldots, n
\end{equation}


![bmwflc-band](fig/bmwflc-band.png)


\begin{exmp}
	\hspace{1px}\\
	To get a better understanding of how finding frequencies in the signal using the magnitude spectrum in the BMWFLC works, we will use the signal $s_k=\underbrace{4}_{A_1}\sin(2\pi \underbrace{4.2 }_{f_1}t) + \underbrace{3}_{A_2}\cos(2\pi \underbrace{5}_{f_2} t)$ as input to our algorithm. We chose the band of [3Hz, 7Hz], with $\Delta f=0.1$. In Figure \ref{fig:BMWFLCmagnitudeSepctrum}, each stem plotted is the magnitude of the Fourier coefficients of its respective FLC (WFLC for the peaks), after running the algorithm for 10 seconds. The two dominant frequencies have red markers, and it's now easy to see which frequencies are dominant.